In [1]:
%load_ext autoreload
%autoreload 2
%load_ext snakeviz
%load_ext line_profiler

# turn off autoreload so that we can use the old model 
# when editing the current project

from coeditor.common import *
import os

os.chdir(proj_root())

In [2]:
from coeditor.retrieval_model import RetrievalEditorModel, AttentionMode, BatchArgs
from coeditor.api import EditPredictionService, QueryRefEditEncoder, BatchArgs, DecodingArgs
from coeditor.dataset import load_datasets
import torch
import copy

In [3]:
model_path = get_model_dir(True) / "coeditor-large-request-stub-v2"
model = RetrievalEditorModel.load(model_path)
model.to("cuda:2")
model.attention_mode = AttentionMode.bidirectional

batch_args = copy.deepcopy(BatchArgs())
batch_args.max_total_ref_tks //= 3
batch_args.min_queires *= 3
batch_args.max_queries *= 2

In [4]:
encoder = QueryRefEditEncoder()
dataset_dir = get_dataset_dir("large") / (repr_modified_args(encoder))
test_data = load_datasets(dataset_dir, ["train"])["train"]
test_edits = test_data.all_edits()[:100]
del test_data

In [7]:
# query_ref_layer not batched
%timeit -n 1 -r 2 model.run_on_edits(test_edits, batch_args)

num batches: 7, batch stats: {'mean': '14.3', 'median': '16.0', 'min': '4.0', 'max': '16.0'}
num batches: 7, batch stats: {'mean': '14.3', 'median': '16.0', 'min': '4.0', 'max': '16.0'}


Training Epoch 0: 100%|██████████| 7/7 [00:10<00:00,  1.57s/it]


num batches: 7, batch stats: {'mean': '14.3', 'median': '16.0', 'min': '4.0', 'max': '16.0'}
num batches: 7, batch stats: {'mean': '14.3', 'median': '16.0', 'min': '4.0', 'max': '16.0'}


Training Epoch 0: 100%|██████████| 7/7 [00:10<00:00,  1.57s/it]

11 s ± 3.62 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [7]:
%timeit -n 1 -r 2 model.run_on_edits(test_edits, batch_args)

num batches: 7, batch stats: {'mean': '14.3', 'median': '16.0', 'min': '4.0', 'max': '16.0'}
num batches: 7, batch stats: {'mean': '14.3', 'median': '16.0', 'min': '4.0', 'max': '16.0'}


Training Epoch 0: 100%|██████████| 7/7 [00:10<00:00,  1.56s/it]


num batches: 7, batch stats: {'mean': '14.3', 'median': '16.0', 'min': '4.0', 'max': '16.0'}
num batches: 7, batch stats: {'mean': '14.3', 'median': '16.0', 'min': '4.0', 'max': '16.0'}


Training Epoch 0: 100%|██████████| 7/7 [00:10<00:00,  1.55s/it]

10.9 s ± 6.59 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [21]:
%snakeviz -t model.run_on_edits(test_edits, batch_args)

num batches: 7, batch stats: {'mean': '14.3', 'median': '16.0', 'min': '9.0', 'max': '16.0'}
num batches: 7, batch stats: {'mean': '14.3', 'median': '16.0', 'min': '4.0', 'max': '16.0'}


Training Epoch 0: 100%|██████████| 7/7 [00:10<00:00,  1.56s/it]


 
*** Profile stats marshalled to file '/tmp/tmpjlbfo3f1'.
Opening SnakeViz in a new tab...
snakeviz web server started on 127.0.0.1:8080; enter Ctrl-C to exit
http://127.0.0.1:8080/snakeviz/%2Ftmp%2Ftmpjlbfo3f1


In [20]:
from coeditor.retrieval_model import t5_sparse_attention

%lprun -T outupt/lprof.txt -f t5_sparse_attention model.run_on_edits(test_edits, batch_args)

num batches: 7, batch stats: {'mean': '14.3', 'median': '16.0', 'min': '9.0', 'max': '16.0'}
num batches: 7, batch stats: {'mean': '14.3', 'median': '16.0', 'min': '4.0', 'max': '16.0'}


Training Epoch 0: 100%|██████████| 7/7 [00:11<00:00,  1.58s/it]


*** Profile printout saved to text file 'lprof.txt'. 


Timer unit: 1e-09 s

Total time: 1.80138 s
File: /home/jiayi/Projects/SPOT/src/coeditor/retrieval_model.py
Function: t5_sparse_attention at line 1220

Line #      Hits         Time  Per Hit   % Time  Line Contents
  1220                                           def t5_sparse_attention(
  1221                                               self: T5Attention,
  1222                                               hidden_states: Tensor,
  1223                                               block_lens: Sequence[int],
  1224                                               get_bias: Callable[[int, int, bool], Tensor],
  1225                                           ):
  1226                                               """
  1227                                               A block-sparse self-attention layer that allows all blocks to attend to the
  1228                                               last block (in additon to themselves).
  1229                                               ""